In [1]:
import pprint
import csv
import ais
import tensorflow as tf

pp = pprint.PrettyPrinter(indent=4)

In [2]:
# Fields
primaryFields = ['TIMESTAMP','TIME','MESSAGE_ID','MMSI','LAT','LONG','POS_ACCURACY','NAV_STATUS','SOG','COG','TRUE_HEADING','RAW_MESSAGE','FISHING_STATUS']
primaryFields = ['TIMESTAMP', 'RAW_MESSAGE', 'FISHING_STATUS']

payloadFields = ['slot_increment', 'sync_state', 'true_heading', 'sog', 'slots_to_allocate', 'rot', 'nav_status', 'repeat_indicator', 'raim', 'id', 'spare', 'keep_flag', 'cog', 'timestamp', 'y', 'x', 'position_accuracy', 'rot_over_range', 'mmsi', 'special_manoeuvre', 'slot_timeout', 'slot_offset', 'slot_number', 'received_stations', 'utc_spare', 'utc_min', 'utc_hour', 'unit_flag', 'spare2', 'mode_flag', 'm22_flag', 'commstate_cs_fill', 'commstate_flag', 'display_flag', 'dsc_flag', 'band_flag', 'gnss', 'fix_type', 'type_and_cargo', 'spare3', 'dim_d', 'assigned_mode', 'dim_b', 'dim_c', 'dim_a', 'name', 'dte']

fieldnames =  ['TIMESTAMP', 'CHECKSUM'] + payloadFields

In [3]:
# Open a new csv file to write out cleaned data
outputFile = open('data/clean_testing_data.csv', 'wb')
writer = csv.DictWriter(outputFile, fieldnames=fieldnames)
writer.writeheader()

In [4]:
# Read file line by line
with open('data/testing_data.csv') as inputFile:
    reader = csv.DictReader(inputFile)
    
    # 'row' is a dictionary of data, keyed by
    # the values in the first row of the csv file.
    for row in reader:

        rawMsg = row['RAW_MESSAGE']
        rawMsgElms = rawMsg.split(',')
        
        for i in range(len(rawMsgElms)) :
            # Add generic fields 1-5
            if i + 1 <= 5 :
                row['F' + str(i + 1)] = rawMsgElms[i]

            # Decode and add payload
            if i + 1 == 6 :
                decodedMsg = ais.decode(rawMsgElms[i], 0)
                for k in decodedMsg.keys() :
                    row[k] = decodedMsg[k]
            
            # Checksum using field 7
            if i + 1 == 7 :
                checksum = 0;
                m = rawMsg[1:-3]
                for j in range(len(m)) :
                    checksum = checksum ^ ord(m[j]);
                checksum = format(checksum, '02X')
                
                if checksum != rawMsgElms[i][2:] :
                    print('checksum did not match')
                    print(rawMsg)
                    row['CHECKSUM'] = 0
                else :
                    row['CHECKSUM'] = 1
        
        data = {}
        for k in fieldnames :
            if k in row :
                data[k] = row[k]
            else :
                data[k] = ''

        writer.writerow(data)
#         pp.pprint(row)

print('----')

checksum did not match
AIVDM,1,1,,,13f:86002Evq;HlRIf2U=T?f0<00,0*7E
checksum did not match
AIVDM,1,1,,A,13gni>001cNo?c<Rs<QP7@8H0@7c,0*56
checksum did not match
AIVDM,1,1,,A,33qjmv8AUwvl29NRl4AP01Lrs000,0*45
checksum did not match
AIVDM,1,1,,A,33A6580siUvncBPRFHAbKpDF0000,0*0C
checksum did not match
AIVDM,1,1,,,33tqK<001gvj?5jS29mJfpJF211h,0*00
checksum did not match
AIVDM,1,1,,B,33gni>0P@;NoLJbRfwTQKl:20141,0*48
checksum did not match
AIVDM,1,1,,B,34`h4<002<vj;JlRspW<?IgN2000,0*4A
checksum did not match
AIVDM,1,1,,,13ok<B0P1KNugOLRS1I<Vr=d08Qb,0*59
checksum did not match
AIVDM,1,1,,A,13gni>001cNo?e>Rs>hh:P6v0@CD,0*2F
checksum did not match
AIVDM,1,1,,B,13cICd001pNpbJDRP5O3hRwF05ad,0*79
checksum did not match
AIVDM,1,1,,A,33tqK<0PATvp`nPRwV:JkpMH2000,0*66
checksum did not match
AIVDM,1,1,,A,13tqK<001TvoQ<lS05>JlHPF26jt,0*21
checksum did not match
AIVDM,1,1,,B,14`h4<002<vo3cpR`v=<CIdN2D00,0*04
checksum did not match
AIVDM,1,1,,B,14`?2r002JvmQh`RfO><Eat<8@5g,0*3A
checksum did not match


In [5]:
outputFile.close()